# Transfer Learning MNIST

* Train a simple convnet on the MNIST dataset the first 5 digits [0..4].
* Freeze convolutional layers and fine-tune dense layers for the classification of digits [5..9].

## 1. Import necessary libraries for the model

In [ ]:
#Importing important modules
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
import tensorflow as tf

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 2. Import MNIST data and create 2 datasets with one dataset having digits from 0 to 4 and other from 5 to 9 

In [ ]:
%matplotlib inline
# Load/Prep the Data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
split_condition1 = y_train <5
split_condition2 = y_train >=5
# labels = y_train[[y_train[5]]
x_train_Ind04 = x_train [split_condition1]
x_test_Ind04 = x_test [y_test<5]
y_train_Ind04 = y_train[split_condition1]
y_test_Ind04 = y_test[y_test<5]

x_train_Ind59 = x_train [split_condition2]
x_test_Ind59 = x_test [y_test>=5]
y_train_Ind59 = y_train[split_condition2]
y_test_Ind59 = y_test[y_test>=5]
#mnist_df = mnist.load_data()
#split1, split2 = tf.split(value, num_or_size_splits=2, axis=1)
# df_1 = 
# df_2 = 
# x_train = x_train.reshape(x_train.shape[0], 28, 28, 1).astype('float32') #
# x_test = x_test.reshape(x_test.shape[0], 28, 28, 1).astype('float32')
# x_train /= 255 # normalization since we have max of 255
# x_test /= 255
# y_train = np_utils.to_categorical(y_train_num, 10) # since it is a classification problem we are explicitly mentioning this
# y_test = np_utils.to_categorical(y_test_num, 10)

# print('--- THE DATA ---')
# print('x_train shape:', x_train.shape)
# print(x_train.shape[0], 'train samples')
# print(x_test.shape[0], 'test samples')

In [ ]:
#y_train

## 3. Print x_train, y_train, x_test and y_test for both the datasets

In [ ]:
print('x_train after 1st half:',x_train_Ind04)
print('y_train after 1st half',y_train_Ind04)
print('x_train after 2nd half',x_train_Ind59)
print('x_train after 2nd half',y_train_Ind59)

print('x_test after 1st half',x_test_Ind04)
print('y_test after 1st half ',y_test_Ind04)
print('x_test after 2nd half ',x_test_Ind59)
print('x_test after 2nd half ',y_test_Ind59)

## ** 4. Let us take only the dataset (x_train, y_train, x_test, y_test) for Integers 0 to 4 in MNIST **
## Reshape x_train and x_test to a 4 Dimensional array (channel = 1) to pass it into a Conv2D layer

In [ ]:
print(x_train_Ind04.shape)
print(y_train_Ind04.shape)
print(x_test_Ind04.shape)
print(y_test_Ind04.shape)

In [ ]:
import numpy as np
x_train_Ind04 = np.reshape(x_train_Ind04, (30596, 28, 28,1))
print(x_train_Ind04.shape)
x_test_Ind04 = np.reshape(x_test_Ind04, (5139, 28, 28,1))
print(x_test_Ind04.shape)

## 5. Normalize x_train and x_test by dividing it by 255

In [ ]:
x_train_Ind04_Normalized = x_train_Ind04/255
print(x_train_Ind04_Normalized)
x_test_Ind04_Normalized = x_test_Ind04/255
print(x_test_Ind04_Normalized)

## 6. Use One-hot encoding to divide y_train and y_test into required no of output classes

In [ ]:
# convert class vectors to binary class matrices using one hot encoding
# y_train_Ind04_Onehot = keras.utils.to_categorical(y_train_Ind04)
# print(y_train_Ind04_Onehot)
# y_test_Ind04_Onehot = keras.utils.to_categorical(y_test_Ind04)
# print(y_test_Ind04_Onehot)

# We use get dummies because when to categorical is used for one hot encoding, it takes from 0 to 4 and 5 to 9 instead of all the way from 0 to 9  even after splitting into 2 halves.
# so we go for get dummies and not to categorical
import pandas as pd
y_train_Ind04_Onehot = pd.get_dummies(y_train_Ind04)
print(y_train_Ind04_Onehot)
y_test_Ind04_Onehot = pd.get_dummies(y_test_Ind04)
print(y_test_Ind04_Onehot)

## 7. Build a sequential model with 2 Convolutional layers with 32 kernels of size (3,3) followed by a Max pooling layer of size (2,2) followed by a drop out layer to be trained for classification of digits 0-4  

In [ ]:
from keras.datasets import cifar10, mnist
from keras.models import Sequential# sequential is one type of model; there are graph models as well
from keras.layers import Dense, Activation, Dropout, Flatten, Reshape # Dense is fully connected layer
from keras.layers import Convolution2D, MaxPooling2D #
from keras.utils import np_utils
import pickle # serialization fancy word for storing on disk
from matplotlib import pyplot as plt
import seaborn as sns
plt.rcParams['figure.figsize'] = (15, 8)

In [ ]:
TRAIN = False
BATCH_SIZE = 32
EPOCHS = 10 

# Define the Type of Model
model2 = Sequential()
# 1st Conv Layer
model2.add(Convolution2D(32, 3, 3, input_shape=(28, 28,1)))
model2.add(Activation('relu'))

# 2nd Conv Layer
model2.add(Convolution2D(32, 3, 3))# input shape is known in the previous layer. so we don't give input shape
model2.add(Activation('relu'))

# Fully Connected Layer
model2.add(Flatten())
model2.add(Dense(128))
model2.add(Activation('relu'))


# Layer 1
model2.add(Dense(output_dim=5, bias=True))
model2.add(Activation("softmax"))

# Loss and Optimizer
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Store Training Results
early_stopping = keras.callbacks.EarlyStopping(monitor='val_acc', patience=5, verbose=1, mode='auto')
callback_list = [early_stopping]# [stats, early_stopping]

# Train the model
model2.fit(x_train_Ind04_Normalized, y_train_Ind04_Onehot, nb_epoch=EPOCHS, batch_size=BATCH_SIZE,
        validation_data=(x_test_Ind04_Normalized, y_test_Ind04_Onehot), callbacks=callback_list, verbose=True)

## 8. Post that flatten the data and add 2 Dense layers with 128 neurons and neurons = output classes with activation = 'relu' and 'softmax' respectively. Add dropout layer inbetween if necessary  

## 9. Print the training and test accuracy

In [ ]:
model2.evaluate(x_train_Ind04, y_train_Ind04_Onehot)

In [ ]:
model2.evaluate(x_test_Ind04, y_test_Ind04_Onehot)

## 10. Make only the dense layers to be trainable and convolutional layers to be non-trainable

In [ ]:
#Freezing layers in the model which don't have 'dense' in their name
for layer in model2.layers:
  if('dense' not in layer.name): #prefix detection to freeze layers which does not have dense
    #Freezing a layer
    layer.trainable = False # freezing the layer

#Module to print colourful statements
from termcolor import colored

#Check which layers have been frozen 
for layer in model2.layers:
  print (colored(layer.name, 'blue'))
  print (colored(layer.trainable, 'red'))

## 11. Use the model trained on 0 to 4 digit classification and train it on the dataset which has digits 5 to 9  (Using Transfer learning keeping only the dense layers to be trainable)

In [ ]:
# repeat all the pre-processing steps for 5 to 9 like in 0 to 4 and then use the 0 to 4 model's learning via transfer learning to 5 to 9 data set
print(x_train_Ind59.shape)
print(y_train_Ind59.shape)
print(x_test_Ind59.shape)
print(y_test_Ind59.shape)

In [ ]:
import numpy as np
x_train_Ind59 = np.reshape(x_train_Ind59,(x_train_Ind59.shape[0], 28, 28,1))
print(x_train_Ind59.shape)
x_test_Ind59 = np.reshape(x_test_Ind59,(x_test_Ind59.shape[0], 28, 28,1))
print(x_test_Ind59.shape)

In [ ]:
# Normalizing the data
x_train_Ind59_Normalized = x_train_Ind59/255
print(x_train_Ind59_Normalized)
x_test_Ind59_Normalized = x_test_Ind59/255
print(x_test_Ind59_Normalized)

In [ ]:
# We use get dummies because when to categorical is used for one hot encoding, it takes from 0 to 4 and 5 to 9 instead of all the way from 0 to 9  even after splitting into 2 halves.
# so we go for get dummies and not to categorical
import pandas as pd
y_train_Ind59_Onehot = pd.get_dummies(y_train_Ind59)
print(y_train_Ind59_Onehot)
y_test_Ind59_Onehot = pd.get_dummies(y_test_Ind59)
print(y_test_Ind59_Onehot)

In [ ]:

# Train the model
model2.fit(x_train_Ind59_Normalized, y_train_Ind59_Onehot, nb_epoch=EPOCHS, batch_size=BATCH_SIZE,
        validation_data=(x_test_Ind59_Normalized, y_test_Ind59_Onehot), callbacks=callback_list, verbose=True)

## 12. Print the accuracy for classification of digits 5 to 9

In [ ]:
model2.evaluate(x_train_Ind59, y_train_Ind59_Onehot)

In [ ]:
model2.evaluate(x_test_Ind59, y_test_Ind59_Onehot)

## Sentiment analysis <br> 

The objective of the second problem is to perform Sentiment analysis from the tweets data collected from the users targeted at various mobile devices.
Based on the tweet posted by a user (text), we will classify if the sentiment of the user targeted at a particular mobile device is positive or not.

### 13. Read the dataset (tweets.csv) and drop the NA's while reading the dataset

In [ ]:
from google.colab import drive
drive.mount ('/content/drive/') 

In [ ]:
import os
os.chdir('/content/drive/My Drive/Colab Notebooks')

In [ ]:
tweets_df = pd.read_csv('tweets.csv').dropna()

### 14. Preprocess the text and add the preprocessed text in a column with name `text` in the dataframe.

In [ ]:
def preprocess(text):
    try:
        return text.decode('ascii')
    except Exception as e:
        return ""

In [ ]:
tweets_df['text'] = [preprocess(text) for text in tweets_df.tweet_text]

In [ ]:
tweets_df.head(5)

### 15. Consider only rows having Positive emotion and Negative emotion and remove other rows from the dataframe.

In [ ]:
tweets_df = tweets_df[(tweets_df.is_there_an_emotion_directed_at_a_brand_or_product == 'Negative emotion') | (tweets_df.is_there_an_emotion_directed_at_a_brand_or_product =='Positive emotion')]

### 16. Represent text as numerical data using `CountVectorizer` and get the document term frequency matrix

#### Use `vect` as the variable name for initialising CountVectorizer.

In [ ]:
# use CountVectorizer to create document-term matrices from X_train and X_test
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
tweets_df_dtm = vect.fit_transform(tweets_df['text'])


### 17. Find number of different words in vocabulary

In [ ]:
dir(vect)

#### Tip: To see all available functions for an Object use dir

In [ ]:
print(len(vect.get_feature_names()))

### 18. Find out how many Positive and Negative emotions are there.

Hint: Use value_counts on that column

In [ ]:
tweets_df['is_there_an_emotion_directed_at_a_brand_or_product'].value_counts()

### 19. Change the labels for Positive and Negative emotions as 1 and 0 respectively and store in a different column in the same dataframe named 'Label'

Hint: use map on that column and give labels

In [ ]:
tweets_df['Label'] = tweets_df['is_there_an_emotion_directed_at_a_brand_or_product'].map({'Positive emotion':1,'Negative emotion':0})

### 20. Define the feature set (independent variable or X) to be `text` column and `labels` as target (or dependent variable)  and divide into train and test datasets

In [ ]:
X = tweets_df['text']
y = tweets_df['Label']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


## 21. **Predicting the sentiment:**


### Use Naive Bayes and Logistic Regression and their accuracy scores for predicting the sentiment of the given text

In [ ]:
#tweets_df['Label']

In [ ]:
# use default options for CountVectorizer
#vect = CountVectorizer()
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
vect = CountVectorizer(ngram_range=(1,2))

# create document-term matrices
X_train_dtm = vect.fit_transform(X_train)
X_test_dtm = vect.transform(X_test)

# use Naive Bayes to predict the star rating
nb = MultinomialNB()
nb.fit(X_train_dtm, y_train)
y_pred_class = nb.predict(X_test_dtm)

# calculate accuracy
print (accuracy_score(y_test, y_pred_class))

In [ ]:
# import and instantiate a logistic regression model
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()

In [ ]:
# train the model using X_train_dtm
logreg.fit(X_train_dtm, y_train)

In [ ]:
# make class predictions for X_test_dtm
y_pred_class = logreg.predict(X_test_dtm)

In [ ]:
# calculate predicted probabilities for X_test_dtm
y_pred_prob = logreg.predict_proba(X_test_dtm)[:, 1]
y_pred_prob

In [ ]:
# calculate accuracy of class predictions
from sklearn import metrics
metrics.accuracy_score(y_test, y_pred_class)
# calculate accuracy
metrics.accuracy_score(y_test, y_pred_class)

## 22. Create a function called `tokenize_predict` which can take count vectorizer object as input and prints the accuracy for x (text) and y (labels)

In [ ]:
def tokenize_predict(vect):
    x_train_dtm = vect.fit_transform(x_train)
    print('Features: ', x_train_dtm.shape[1])
    x_test_dtm = vect.transform(x_test)
    nb = MultinomialNB()
    nb.fit(x_train_dtm, y_train)
    y_pred_class = nb.predict(x_test_dtm)
    print('Accuracy: ', metrics.accuracy_score(y_test, y_pred_class))

### Create a count vectorizer function which includes n_grams = 1,2  and pass it to tokenize_predict function to print the accuracy score

### Create a count vectorizer function with stopwords = 'english'  and pass it to tokenize_predict function to print the accuracy score

In [ ]:
# use default options for CountVectorizer
#vect = CountVectorizer()
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
vect = CountVectorizer(ngram_range=(1,2),stop_words='english')

# create document-term matrices
X_train_dtm = vect.fit_transform(X_train)
X_test_dtm = vect.transform(X_test)

# use Naive Bayes to predict the star rating
nb = MultinomialNB()
nb.fit(X_train_dtm, y_train)
y_pred_class = nb.predict(X_test_dtm)

# calculate accuracy
print (accuracy_score(y_test, y_pred_class))

### Create a count vectorizer function with stopwords = 'english' and max_features =300  and pass it to tokenize_predict function to print the accuracy score

In [ ]:
# use default options for CountVectorizer
#vect = CountVectorizer()
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
vect = CountVectorizer(ngram_range=(1,2),stop_words='english',max_features=300)

# create document-term matrices
X_train_dtm = vect.fit_transform(X_train)
X_test_dtm = vect.transform(X_test)

# use Naive Bayes to predict the star rating
nb = MultinomialNB()
nb.fit(X_train_dtm, y_train)
y_pred_class = nb.predict(X_test_dtm)

# calculate accuracy
print (accuracy_score(y_test, y_pred_class))

### Create a count vectorizer function with n_grams = 1,2  and max_features = 15000  and pass it to tokenize_predict function to print the accuracy score

In [ ]:
# use default options for CountVectorizer
#vect = CountVectorizer()
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
vect = CountVectorizer(ngram_range=(1,2),stop_words='english',max_features=15000)

# create document-term matrices
X_train_dtm = vect.fit_transform(X_train)
X_test_dtm = vect.transform(X_test)

# use Naive Bayes to predict the star rating
nb = MultinomialNB()
nb.fit(X_train_dtm, y_train)
y_pred_class = nb.predict(X_test_dtm)

# calculate accuracy
print (accuracy_score(y_test, y_pred_class))

### Create a count vectorizer function with n_grams = 1,2  and include terms that appear at least 2 times (min_df = 2)  and pass it to tokenize_predict function to print the accuracy score

In [ ]:
# use default options for CountVectorizer
#vect = CountVectorizer()
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
vect = CountVectorizer(ngram_range=(1,2),min_df=2)

# create document-term matrices
X_train_dtm = vect.fit_transform(X_train)
X_test_dtm = vect.transform(X_test)

# use Naive Bayes to predict the star rating
nb = MultinomialNB()
nb.fit(X_train_dtm, y_train)
y_pred_class = nb.predict(X_test_dtm)

# calculate accuracy
print (accuracy_score(y_test, y_pred_class))